# DAVID mk21 - 마감일 텍스트 수정 및 레이아웃 개선
- 입력 마감일 → **마감일**로 변경
- 검정색 기준선 오른쪽 콘텐츠 폭 증가
- 전체 UI 정렬 깔끔하게 개선


In [ ]:
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime
import json

# 데이터 저장용
todo_items = []

# 레이아웃
layout_label = widgets.Layout(width='80px')
layout_input = widgets.Layout(width='300px')
layout_button = widgets.Layout(width='100px', height='36px')
layout_full = widgets.Layout(width='100%')

def update_overall_progress():
    total = len(todo_items)
    done = sum(i['progress'] for i in todo_items)
    if total > 0:
        progress_bar.value = done / total
        progress_label.value = f"({int(progress_bar.value * 100)}%)"
    else:
        progress_bar.value = 0
        progress_label.value = "(0%)"

def refresh_list():
    box_list.children = []
    for item in todo_items:
        # 좌측
        left = widgets.VBox([
            widgets.Label("할 일", layout=layout_label),
            widgets.Label("진행률", layout=layout_label),
            widgets.Label("마감일", layout=layout_label),
            widgets.Label("중요도", layout=layout_label),
            widgets.Label("작업", layout=layout_label),
        ])
        # 우측
        title = widgets.Text(value=item['title'], layout=layout_input)
        bar = widgets.FloatProgress(value=item['progress'], min=0, max=100, bar_style='info', layout=layout_input)
        slider = widgets.IntSlider(value=item['progress'], min=0, max=100, step=5, layout=layout_input)
        date = widgets.Label(value=item['due'], layout=layout_input)
        priority = widgets.Label(value=item['priority'], layout=layout_input)
        complete = widgets.Button(description='완료✅', layout=widgets.Layout(width='120px'))
        delete = widgets.Button(description='삭제❌', layout=widgets.Layout(width='120px'))

        complete_box = widgets.VBox([complete, delete], layout=layout_input)

        def on_change(change, i=item):
            i['progress'] = change['new']
            bar.value = change['new']
            update_overall_progress()

        slider.observe(on_change, names='value')

        def on_delete(b, i=item):
            todo_items.remove(i)
            refresh_list()
            update_overall_progress()

        def on_complete(b, i=item):
            i['progress'] = 100
            refresh_list()
            update_overall_progress()

        delete.on_click(on_delete)
        complete.on_click(on_complete)

        right = widgets.VBox([title, bar, slider, date, priority, complete_box])
        row = widgets.HBox([left, widgets.Box([right], layout=widgets.Layout(border='2px solid black', padding='6px'))])
        box_list.children += (row,)

# 입력 위젯
title_input = widgets.Text(placeholder="할 일을 입력하세요", layout=layout_input)
progress_input = widgets.IntSlider(min=0, max=100, step=5, layout=layout_input)
due_input = widgets.DatePicker(layout=layout_input)
priority_input = widgets.Dropdown(options=['낮음', '보통', '높음'], layout=layout_input)
add_btn = widgets.Button(description='추가', button_style='success', layout=widgets.Layout(width='100%', height='40px'))

def on_add(b):
    if title_input.value:
        todo_items.append({
            'title': title_input.value,
            'progress': progress_input.value,
            'due': due_input.value.strftime('%Y-%m-%d') if due_input.value else '',
            'priority': priority_input.value
        })
        title_input.value = ''
        progress_input.value = 0
        due_input.value = None
        priority_input.value = '보통'
        refresh_list()
        update_overall_progress()

add_btn.on_click(on_add)

# 저장/불러오기
def save_data(_):
    with open("todo_data.json", "w") as f:
        json.dump(todo_items, f)

def load_data(_):
    try:
        with open("todo_data.json", "r") as f:
            data = json.load(f)
            todo_items.clear()
            todo_items.extend(data)
            refresh_list()
            update_overall_progress()
    except:
        print("불러오기 실패")

save_btn = widgets.Button(description="저장", button_style='info', layout=layout_button)
load_btn = widgets.Button(description="불러오기", button_style='warning', layout=layout_button)
save_btn.on_click(save_data)
load_btn.on_click(load_data)

progress_bar = widgets.FloatProgress(min=0, max=1.0, value=0.0, layout=widgets.Layout(width='300px'))
progress_label = widgets.Label(value="(0%)")

# 출력
box_list = widgets.VBox()
display(widgets.VBox([
    widgets.HTML("<h3>DAVID mk21 - 마감일 텍스트 수정 및 우측 폭 조정</h3>"),
    widgets.HBox([widgets.Label("할 일", layout=layout_label), title_input]),
    widgets.HBox([widgets.Label("진행률", layout=layout_label), progress_input]),
    widgets.HBox([widgets.Label("마감일", layout=layout_label), due_input]),
    widgets.HBox([widgets.Label("중요도", layout=layout_label), priority_input]),
    add_btn,
    widgets.HBox([save_btn, load_btn]),
    widgets.HBox([progress_bar, progress_label]),
    box_list
]))

refresh_list()
update_overall_progress()
